In [ ]:
%pip install kubernetes

In [88]:
from kubernetes import client, config
import json
config.load_kube_config()

v1 = client.CoreV1Api()

In [89]:
# https://github.com/kubernetes-client/python/blob/master/kubernetes/utils/quantity.py
from kubernetes.utils import quantity
quantity.parse_quantity("40Mi")
quantity.parse_quantity("3m")

Decimal('0.003')

In [120]:
def get_pod_usage(containers):
  memory = 0
  cpu = 0
  if isinstance(containers, list) and isinstance(containers[0], dict):
    assert len(containers) <= 2, "Only 2 containers allowed"
    for c in containers:
      cpu += quantity.parse_quantity(c['usage']['cpu'])
      memory += quantity.parse_quantity(c['usage']['memory'])
  else:
    for c in containers:
      limits = c.resources.limits
      if limits is None:
        continue
      cpu += quantity.parse_quantity(limits['cpu'])
      memory += quantity.parse_quantity(limits['memory'])
  return dict(cpu = cpu, memory = memory)

In [91]:
# kubectl get --raw "/apis/metrics.k8s.io/v1beta1/nodes"
# https://kubernetes.io/docs/tasks/debug/debug-cluster/resource-metrics-pipeline/
api = client.CustomObjectsApi()
resource = api.list_namespaced_custom_object(group="metrics.k8s.io",version="v1beta1", namespace="default", plural="pods")


In [126]:
# resource return by `list_namespaced_custom_object` is a dict
for pod in resource["items"]:
  pod_name = pod['metadata']['name']
  # target_pod is a Pod object
  target_pod = v1.read_namespaced_pod(pod_name, namespace='default')
  limits = get_pod_usage(target_pod.spec.containers)
  usage = get_pod_usage(pod['containers'])
  cpu_usage = usage['cpu'] * 1000
  memory_usage = usage['memory'] / 1024 / 1024
  limit_cpu = limits['cpu'] * 1000
  limit_memory = limits['memory']  / 1024 / 1024
  print(f"name: {pod_name}, cpu: {cpu_usage}/{limit_cpu}, memory: {memory_usage}/{limit_memory} " )
  print(f"name: {pod_name}, cpu: {round(cpu_usage)}, memory: {round(memory_usage)} " )
  print(f"percent: {cpu_usage / limit_cpu}, {memory_usage /limit_memory}")

name: counter, cpu: 3.089643000/2000, memory: 36.328125/1024 
name: counter, cpu: 3, memory: 36 
percent: 0.0015448215, 0.0354766845703125
name: details-v1-5498c86cf5-4wxd7, cpu: 2.100719000/2000, memory: 49.51171875/1024 
name: details-v1-5498c86cf5-4wxd7, cpu: 2, memory: 50 
percent: 0.0010503595, 0.048351287841796875
name: httpbin-85d76b4bb6-5fptx, cpu: 3.402389000/2000, memory: 70.578125/1024 
name: httpbin-85d76b4bb6-5fptx, cpu: 3, memory: 71 
percent: 0.0017011945, 0.0689239501953125
name: httpbin-85d76b4bb6-k86j4, cpu: 2.501122000/2000, memory: 70.75/1024 
name: httpbin-85d76b4bb6-k86j4, cpu: 3, memory: 71 
percent: 0.001250561, 0.069091796875
name: productpage-v1-65b75f6885-96hl4, cpu: 9.420458000/2000, memory: 76.578125/1024 
name: productpage-v1-65b75f6885-96hl4, cpu: 9, memory: 77 
percent: 0.004710229, 0.0747833251953125
name: ratings-v1-b477cf6cf-9zr5n, cpu: 2.651078000/2000, memory: 39.94140625/1024 
name: ratings-v1-b477cf6cf-9zr5n, cpu: 3, memory: 40 
percent: 0.0013255

In [93]:
resource["items"][0]

{'metadata': {'name': 'counter',
  'namespace': 'default',
  'creationTimestamp': '2022-05-12T03:23:18Z',
  'labels': {'security.istio.io/tlsMode': 'istio',
   'service.istio.io/canonical-name': 'counter',
   'service.istio.io/canonical-revision': 'latest'}},
 'timestamp': '2022-05-12T03:23:12Z',
 'window': '15.927s',
 'containers': [{'name': 'istio-proxy',
   'usage': {'cpu': '2171530n', 'memory': '35536Ki'}},
  {'name': 'count', 'usage': {'cpu': '918113n', 'memory': '1664Ki'}}]}

In [122]:
# Resources set by user
# Pod is Pod object
pod_name = resource["items"][0]['metadata']['name']
pod = v1.read_namespaced_pod(pod_name, namespace='default')
get_pod_usage(pod.spec.containers)
# pod.spec.containers[0].resources

{'cpu': Decimal('2'), 'memory': Decimal('1073741824')}

In [95]:
usage = get_pod_usage(resource["items"][0])
cpu_usage = usage['cpu'] * 1000
memory_usage = usage['memory'] / 1024 / 1024
print(f"name: {pod_name}, cpu: {cpu_usage}, memory: {memory_usage} " )
print(f"name: {pod_name}, cpu: {round(cpu_usage)}, memory: {round(memory_usage)} " )

name: counter, cpu: 3.089643000, memory: 36.328125 
name: counter, cpu: 3, memory: 36 


In [115]:
ret = v1.list_namespaced_pod(namespace='default')
ret.items[0].spec.containers[-1].resources.limits

{'cpu': '2', 'memory': '1Gi'}